In [52]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import json
import os
import pandas as pd
pd.set_option("display.max_columns",30)  # Shows all of the columns
import requests

In [53]:
"""
1. get the data from S3
2. weather_data transoformation 
3. taxi_trips transformations - DONE
4. update payment_type_master - DONE
5. update company_master - DONE
6. update taxi_trips with company and payment_type ids (replace the string values with ids from the latest master tables)
7. upload weather data to s3
8. upload taxi data to s3
9. upload the newest payment_type_master and company_master 
""" 

'\n1. get the data from S3\n2. weather_data transoformation\n3. taxi_trips transformations\n4. update payment_type_master\n5. update company_master\n6. update taxi_trips with company and payment_type ids (replace the string values with ids from the latest master tables)\n7. upload weather data to s3\n8. upload taxi data to s3\n9. upload the newest payment_type_master and company_master \n'

### taxi_trips transformation

In [54]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")


url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"

headers = {"X-APP-Token": os.environ.get("CHICHAGO_API_TOKEN")}

response = requests.get(url)

data = response.json()

In [55]:
taxi_trips = pd.DataFrame(data)

In [56]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

taxi_trips.dropna(inplace=True)

taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                           "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("H")



In [57]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,00c6d0cc3d95795a2b9cb3fcbb3afa8eed897ac7,3ae83cc261cea27eafc3d9b18bbc93100c03762e8b6971...,2024-03-15T23:45:00.000,2024-03-16T00:15:00.000,1680,1,76,32,42.25,9.55,0,5,56.8,Credit Card,Taxi Affiliation Services,41.97907082,-87.903039661,41.884987192,-87.620992913,2024-03-15 23:00:00
1,fd729619c5458746ffe602406d2f86cb4059ee35,3c814d3baedca9be4de8ddb5547c7dec404a08e148740e...,2024-03-15T23:45:00.000,2024-03-16T00:15:00.000,1569,18.06,76,8,44.5,14.7,0,4,63.7,Credit Card,Sun Taxi,41.980264315,-87.913624596,41.899602111,-87.633308037,2024-03-15 23:00:00
2,f8aef1517ec3953a3805d0912f92f994e1245205,9de14279ac4dc5696c73c13b07b0aaf2b1a1796dda9f4c...,2024-03-15T23:45:00.000,2024-03-15T23:45:00.000,336,1.64,22,24,10,2,0,0,12,Mobile,City Service,41.92276062,-87.699155343,41.901206994,-87.676355989,2024-03-15 23:00:00
3,efa623a2405100c731e62efc223d3c02ddee3d8a,8a999732f0972dda5aa358ad377427f0cb844b5ec246a9...,2024-03-15T23:45:00.000,2024-03-16T00:00:00.000,1370,5.51,38,8,18.5,0,0,1,19.5,Prcard,Flash Cab,41.812948939,-87.617859676,41.899602111,-87.633308037,2024-03-15 23:00:00
4,ebbf769b25db8056d8ffc0b27f982ac1102eab4e,7ff3ea8c15d902e432f0f3ca3aab1d5f20bff4c4fedfb5...,2024-03-15T23:45:00.000,2024-03-15T23:45:00.000,600,2.3,6,8,9,2.1,0,1,12.1,Credit Card,Taxi Affiliation Services,41.944226601,-87.655998182,41.899602111,-87.633308037,2024-03-15 23:00:00


#### taxi_trips transformation function

In [58]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """" Perform transformation with the taxi data.
    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The DataFrame holding the daily taxi trips

    Returns
    -------
    pd.DataFrame
        The cleaned, transformed DataFrame holding the daily taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas DataFrame")

    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract",
                     "pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                                "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("H")

    return taxi_trips

### copmany update codes

In [59]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
        "company_id": range(1, len(company_master)+ 1),
        "company": company_master
    }

)

company_master.tail()

,company_id,company
25,26,Koam Taxi Association
26,27,Setare Inc
27,28,6574 - Babylon Express Inc.
28,29,5167 - 71969 5167 Taxi Inc
29,30,3556 - 36214 RC Andrews Cab


In [60]:
new_company_data = [
    {"company": "6574 - Babylon Express Inc."},
    {"company": "X"},
    {"company": "Y"}
]

new_company_mapping = pd.DataFrame(new_company_data)

new_company_mapping

,company
0,6574 - Babylon Express Inc.
1,X
2,Y


In [61]:
company_max_id = company_master["company_id"].max()
company_max_id

30

In [62]:
new_companies_list = []

for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        new_companies_list.append(company)

# one line
new_companies_list = [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]

new_companies_list

['X', 'Y']

In [63]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
    "company": new_companies_list
})

new_company_df

,company_id,company
0,31,X
1,32,Y


In [64]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

updated_company_master.tail()

,company_id,company
27,28,6574 - Babylon Express Inc.
28,29,5167 - 71969 5167 Taxi Inc
29,30,3556 - 36214 RC Andrews Cab
30,31,X
31,32,Y


In [65]:
def updated_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """Extend the company master with new companies if there are new companies

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily taxi trips
    company_master : pd.DataFrame

    Returns
    -------
    pd.DataFrame
        The updated company_master data, if new companies are in the taxi data, they will be loaded to it.
    """

    company_max_id = company_master["company_id"].max()
    new_companies_list = [company for company in taxi_trips["company"].values if company not in company_master["company"].values]
    new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
    "company": new_companies_list
    })

    new_companies_df

    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [68]:
taxi_trips_company_only = pd.DataFrame({
    "company_id": [1,2,3],
    "company": ["6574 - Babylon Express Inc.", "X", "Y"]
})

taxi_trips_company_only

,company_id,company
0,1,6574 - Babylon Express Inc.
1,2,X
2,3,Y


In [73]:
updated_company_master = updated_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [74]:
updated_company_master.tail()

,company_id,company
27,28,6574 - Babylon Express Inc.
28,29,5167 - 71969 5167 Taxi Inc
29,30,3556 - 36214 RC Andrews Cab
30,31,X
31,32,Y


### payment_type_master codes

In [81]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1, len(payment_type_master)+ 1),
        "payment_type": payment_type_master
    }

)

taxi_trips_payment_type_only = pd.DataFrame({
    "payment_type_id": [1,2,3],
    "payment_type": ["Credit Card", "X", "Y"]
})

taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [86]:
def updated_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    """Extend the payment type master with new payment types if there are new payment types

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily taxi trips
    payment_type_master : pd.DataFrame
        Dataframe holding the payment_type_master data.

    Returns
    -------
    pd.DataFrame
        The updated payment_type_master data, if new payment types are in the taxi data, they will be loaded to it.
    """

    payment_type_max_id = payment_type_master["payment_type_id"].max()

    new_payment_type_list = [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
    new_payment_type_df = pd.DataFrame({
    "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_list) + 1),
    "payment_type": new_payment_type_list
    })

    

    updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

    return updated_payment_type_master

In [87]:
updated_payment_type_master = updated_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [88]:
updated_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Prcard
3,4,Cash
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


### Creating a generic update master table function

In [90]:
def updated_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    """Extend master DataFrame with new values if there are any.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame holding the daily taxi trips
    master : pd.DataFrame
        Dataframe holding the payment_type_master data.
    id_column : str
        The id column of the master DataFrame
    value_column : str
         The name of the column in master_df containing the values.   

    Returns
    -------
    pd.DataFrame
        The updated master data, if new values are in the taxi data, they will be loaded to it.
    """

    max_id = master[id_column].max()

    new_values_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
    new_values_df = pd.DataFrame({
        id_column: range(max_id + 1, max_id + len(new_values_list) + 1),
        value_column: new_values_list
    })

    
    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master

In [93]:
test_payment_type_master = updated_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master, id_column="payment_type_id", value_column="payment_type")

In [94]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Prcard
3,4,Cash
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


In [95]:
test_company_master = updated_master(taxi_trips=taxi_trips_company_only, master=company_master, id_column="company_id", value_column="company")

In [96]:
test_company_master.tail()

,company_id,company
27,28,6574 - Babylon Express Inc.
28,29,5167 - 71969 5167 Taxi Inc
29,30,3556 - 36214 RC Andrews Cab
30,31,X
31,32,Y


### update taxi_trips with the most recent company_master and payment_type master codes

In [97]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,00c6d0cc3d95795a2b9cb3fcbb3afa8eed897ac7,3ae83cc261cea27eafc3d9b18bbc93100c03762e8b6971...,2024-03-15T23:45:00.000,2024-03-16T00:15:00.000,1680,1,76,32,42.25,9.55,0,5,56.8,Credit Card,Taxi Affiliation Services,41.97907082,-87.903039661,41.884987192,-87.620992913,2024-03-15 23:00:00
1,fd729619c5458746ffe602406d2f86cb4059ee35,3c814d3baedca9be4de8ddb5547c7dec404a08e148740e...,2024-03-15T23:45:00.000,2024-03-16T00:15:00.000,1569,18.06,76,8,44.5,14.7,0,4,63.7,Credit Card,Sun Taxi,41.980264315,-87.913624596,41.899602111,-87.633308037,2024-03-15 23:00:00
2,f8aef1517ec3953a3805d0912f92f994e1245205,9de14279ac4dc5696c73c13b07b0aaf2b1a1796dda9f4c...,2024-03-15T23:45:00.000,2024-03-15T23:45:00.000,336,1.64,22,24,10,2,0,0,12,Mobile,City Service,41.92276062,-87.699155343,41.901206994,-87.676355989,2024-03-15 23:00:00
3,efa623a2405100c731e62efc223d3c02ddee3d8a,8a999732f0972dda5aa358ad377427f0cb844b5ec246a9...,2024-03-15T23:45:00.000,2024-03-16T00:00:00.000,1370,5.51,38,8,18.5,0,0,1,19.5,Prcard,Flash Cab,41.812948939,-87.617859676,41.899602111,-87.633308037,2024-03-15 23:00:00
4,ebbf769b25db8056d8ffc0b27f982ac1102eab4e,7ff3ea8c15d902e432f0f3ca3aab1d5f20bff4c4fedfb5...,2024-03-15T23:45:00.000,2024-03-15T23:45:00.000,600,2.3,6,8,9,2.1,0,1,12.1,Credit Card,Taxi Affiliation Services,41.944226601,-87.655998182,41.899602111,-87.633308037,2024-03-15 23:00:00


In [110]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master, company_master: pd.DataFrame) -> pd.DataFrame:
    """Update the taxi_trips DataFrame with the company_master and the payment_type_master ids and delete the string columns

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The DataFrame with the daily taxi trips.
    payment_type_master : pd.DataFrame
        The payment type master table.
    company_master : pd.DataFrame
        The company master table.

    Returns
    -------
    pd.DataFrame
        The taxi trips data with only payment_type_id and company_ide, without company or payment_type values
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")

    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)

    return taxi_trips_id

In [111]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)

taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
18515,a0fc70f7efd1c84fb051384e27529f52890bbd67,9a423881d4046f5590943c08ec5e58d59f5ebd91fe26aa...,2024-03-15T17:45:00.000,2024-03-15T18:15:00.000,1162,2.74,8,6,11.66,1.53,0,0,13.19,41.899602111,-87.633308037,41.944226601,-87.655998182,2024-03-15 17:00:00,2,10
13214,4243ebcc524c7ba2402fd40fc971a2ec03bd7556,90075a115851ba1b84cb1d85599c04c4d412d0afde5333...,2024-03-15T18:15:00.000,2024-03-15T18:30:00.000,941,6.97,76,10,19.5,0,0,4,23.5,41.980264315,-87.913624596,41.985015101,-87.804532006,2024-03-15 18:00:00,4,5
1440,f4dd224cc02700976d5a76939caa9cdcfbf0e04f,3f68542d52a06c2f726a08b3888acb6e7979a3ade1b74c...,2024-03-15T22:30:00.000,2024-03-15T22:30:00.000,180,0.5,35,34,5,0,0,0,5,41.835117986,-87.618677767,41.842076117,-87.633973422,2024-03-15 22:00:00,4,1
4296,4f95b13780373f4a41755bf0e655081dc3ebf344,99ab99695f31d67888669dcd596cbd0cda61f5c717b8d7...,2024-03-15T00:45:00.000,2024-03-15T00:45:00.000,537,6.85,76,10,18.5,4.6,0,4,27.6,41.980264315,-87.913624596,41.985015101,-87.804532006,2024-03-15 00:00:00,1,2
5571,ae6b0fc6731421175e02389dbfc7fa6429ba4a7f,c3c7eca7dbd2a69ff3195e717334343ed19de792129c84...,2024-03-15T18:15:00.000,2024-03-15T19:15:00.000,3360,17.3,32,8,45.75,13.81,0,9,69.06,41.880994471,-87.632746489,41.890922026,-87.618868355,2024-03-15 18:00:00,1,7
